# An Introduction to LangChain

In this notebook, we will cover the very basics of LangChain. This includes chains and templates.

First, we import necessary libraries, and use `dotenv` to load our OpenAI API key.

In [1]:
from langchain_community.chat_models import ChatOpenAI
import openai
import os

from dotenv import load_dotenv

api_key = os.getenv("OPENAI_API_KEY")

load_dotenv()

True

In [4]:
from langchain.schema import (
    HumanMessage,
    SystemMessage
)

In [7]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=api_key)

In every chat, LLMs are typically first introduced to a "system message," instructing the LLM on how to interpret the conversation. There is also a "human" or "user" message which is simply what the user sends to the LLM. An "assistant" or "AI" message is associated with the messages that the LLM itself writes. 

In [8]:
messages = [
    SystemMessage(content="You are an expert data scientist."),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data. Only return the script.")
]

response = llm(messages=messages, temperature=0.6)
print(response.content)

/tmp/ipykernel_1402/2497210350.py:6: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages=messages, temperature=0.6)


```python
import numpy as np
import tensorflow as tf

# Generate simulated data
np.random.seed(0)
X = np.random.rand(1000, 10)
y = np.random.randint(0, 2, 1000)

# Define the neural network architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32)
```


Note the use of the "temperature" parameter. Temperature is a parameter that refers to the probability with which the LLM's underlying next token predictor picks out a next token that is not the highest probability token. You can consider to it a proxy for "creativity." Higher temperature -> more randomness -> more "creativity."

We represent prompts using "prompt templates," which allow us to dynamically plug things into prompts. The `PromptTemplate` class is simply LangChain's object interface with prompts. Chains allow us to link prompts together. 

In [10]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [12]:
prompt = """
Explain {topic} in one sentence
"""

prompt = PromptTemplate.from_template(prompt)

In [14]:
chain = LLMChain(prompt=prompt, llm=llm)

chain.run(topic="atitude")

'Attitude is the way a person thinks, feels, and behaves towards a particular situation, person, or thing.'

In [37]:
second_prompt = PromptTemplate(
    input_variables=["ml_topic_desc"],
    template="""
    You are a:
    {ml_topic_desc}
    """
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)
chain.run(topic= "vegetable")

'A vegetable is a plant or part of a plant that is consumed by humans as food.'

The outputs of the first chain are passed in to the second chain as input.

In [40]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

blog_post = overall_chain.run(input="vegetable")
print(blog_post)



> Entering new SimpleSequentialChain chain...
Vegetables are edible plants that are typically rich in nutrients and consumed as part of a healthy diet.

They come in a variety of colors, shapes, and sizes, and can be eaten raw or cooked in various dishes. Some common vegetables include carrots, broccoli, spinach, and tomatoes. Vegetables provide essential vitamins, minerals, and fiber that are important for maintaining good health. Including a variety of vegetables in your diet can help reduce the risk of chronic diseases and promote overall well-being. So, if you are a vegetable, you are a nutritious and important part of a balanced diet!

> Finished chain.

They come in a variety of colors, shapes, and sizes, and can be eaten raw or cooked in various dishes. Some common vegetables include carrots, broccoli, spinach, and tomatoes. Vegetables provide essential vitamins, minerals, and fiber that are important for maintaining good health. Including a variety of vegetables in your diet 